In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K
from keras.optimizers import Adam
import os
from IPython.display import display
%autosave 5

Autosaving every 5 seconds


In [2]:
def loadData(dataset,split=0):
    if(split == 0):
        print("In non split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51)
    else:
        print("In split")
        return tf.keras.preprocessing.image_dataset_from_directory(f"../Data/{dataset}",
                                                                   labels='inferred',shuffle=True,seed=51,
                                                            validation_split=split,subset='both')

In [3]:
#from https://aakashgoel12.medium.com/how-to-add-user-defined-function-get-f1-score-in-keras-metrics-3013f979ce0d
def f1_score(y_true, y_pred): #taken from old keras source code 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [4]:
def createModel(convFilters, convAndPoolLayers,numberOfFCLayers, numberOfNeuronsPerFCLayer, numberOfEpochs, dropoutRate):
    model = keras.Sequential()
        
    for convAndPoolLayer in range(convAndPoolLayers):
        model.add(keras.layers.Conv2D(convFilters,(3,3), activation='relu',padding='valid'))
        model.add(keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid'))
        
    model.add(keras.layers.Flatten())
    
    for layer in range(numberOfFCLayers):
        if layer == numberOfFCLayers - 1:
            model.add(keras.layers.Dense(1,activation='softmax'))
        else:
            model.add(keras.layers.Dense(numberOfNeuronsPerFCLayer,activation='relu'))
            model.add(keras.layers.Dropout(dropoutRate))
             
    model.compile(optimizer="adam",loss='binary_crossentropy', metrics=f1_score)
    return model

In [5]:
def createModelParametersDF(n_convFilters,n_convAndPoolLayers,
                            n_FCLayers,n_NeuronsPerFCLayers,dropoutRates,trainScores,devScores):
    modelParameters = dict()
    modelParameters['n_convFilters'] = n_convFilters
    modelParameters['n_convAndPoolLayers'] = n_convAndPoolLayers
    modelParameters['n_FCLayers'] = n_FCLayers
    modelParameters['n_NeuronsPerFCLayers'] = n_NeuronsPerFCLayers
    modelParameters['dropoutRate'] = dropoutRates
    modelParameters['trainScore'] = trainScores
    modelParameters['devScore'] = devScores

    modelParametersDF = pd.DataFrame(modelParameters, columns=modelParameters.keys())
    return modelParametersDF

In [6]:
def createRangeFromMidpoint(midpoint,range,mandatoryMinimum=1):
    possibleMin = int(midpoint-(range/2))
    possibleMin = max([mandatoryMinimum,possibleMin])
    possibleMax = int(midpoint+(range/2))
    possibleRange = np.arange(possibleMin,possibleMax)
    return possibleRange

In [7]:
def main():
    train = loadData("training")
    dev,test = loadData("testing",.5)

    choose = np.random.choice

    possibleConvFilters = createRangeFromMidpoint(32,64)

    possibleConvAndPoolLayers = createRangeFromMidpoint(3,6)

    possibleNumberOfFCLayers = createRangeFromMidpoint(5,10)
    possibleNumberOfNeuronsPerFCLayer = createRangeFromMidpoint(5,10)

    possibleNumberOfEpochs = createRangeFromMidpoint(5,10)

    n_convFilters = []
    n_convAndPoolLayers = []
    n_FCLayers = []
    n_NeuronsPerFCLayers = []
    dropoutRates = []
    trainScores = []
    devScores = []
    
    trial = 0
    while trial < 10:
        convFilters = choose(possibleConvFilters)

        convAndPoolLayers = choose(possibleConvAndPoolLayers)
        convAndPoolLayers = min([convAndPoolLayers,6])

        numberOfFCLayers = choose(possibleNumberOfFCLayers)
        numberOfNeuronsPerFCLayer = choose(possibleNumberOfNeuronsPerFCLayer)

        numberOfEpochs = choose(possibleNumberOfEpochs)
        
        dropoutRate = np.random.rand()

        model = createModel(convFilters, convAndPoolLayers,numberOfFCLayers, numberOfNeuronsPerFCLayer, numberOfEpochs, dropoutRate)
    
        model.fit(train,epochs=numberOfEpochs,verbose=0)

        model_path = f'../Models/FlipDetectionModelTrials/fd_model_{trial}.h5'
        model.save(model_path)
        model_size = os.path.getsize(model_path) / (1024 * 1024)
        if model_size < 40:
            print()
            print('trainScore')
            trainScore = model.evaluate(train)[1]
            print('devScore')
            devScore = model.evaluate(dev)[1]

            n_convFilters.append(convFilters)
            n_convAndPoolLayers.append(convAndPoolLayers)
            n_FCLayers.append(numberOfFCLayers)
            n_NeuronsPerFCLayers.append(numberOfNeuronsPerFCLayer)
            dropoutRates.append(dropoutRate)
            trainScores.append(trainScore)
            devScores.append(devScore)
            print('concluding trial ',trial)
            trial += 1
        else:
            print(f'redoing trial {trial}. Model was {model_size}MB.')
            failedTrial = createModelParametersDF([convFilters],[convAndPoolLayers],
                                               [numberOfFCLayers],[numberOfNeuronsPerFCLayer],[dropoutRate],[np.nan],[np.nan])
            display(failedTrial)
            
    modelParametersDF = createModelParametersDF(n_convFilters,n_convAndPoolLayers,
                                                n_FCLayers,n_NeuronsPerFCLayers,dropoutRates,trainScores,devScores)
    display(modelParametersDF.sort_values(by='trainScore', ascending=False))
        
    
    

In [8]:
main()

In non split
Found 2392 files belonging to 2 classes.
In split
Found 597 files belonging to 2 classes.
Using 299 files for training.
Using 298 files for validation.


2024-06-14 02:08:10.179897: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2392]
	 [[{{node Placeholder/_4}}]]
2024-06-14 02:08:10.180035: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2392]
	 [[{{node Placeholder/_0}}]]
2024-06-14 02:08:10.504454: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz



trainScore
75/75 [==============================] - 10s 130ms/step - loss: 0.6927 - f1_score: 0.6498
devScore


2024-06-14 02:10:20.850130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]
2024-06-14 02:10:20.850275: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [299]
	 [[{{node Placeholder/_4}}]]


10/10 [==============================] - 1s 89ms/step - loss: 0.6919 - f1_score: 0.6277
concluding trial  0

trainScore
75/75 [==============================] - 12s 158ms/step - loss: 0.6928 - f1_score: 0.6493
devScore
10/10 [==============================] - 2s 114ms/step - loss: 0.6921 - f1_score: 0.6244
concluding trial  1

trainScore
75/75 [==============================] - 18s 231ms/step - loss: 0.6929 - f1_score: 0.6492
devScore
10/10 [==============================] - 2s 184ms/step - loss: 0.6926 - f1_score: 0.6384
concluding trial  2

trainScore
75/75 [==============================] - 6s 72ms/step - loss: 0.6928 - f1_score: 0.6493
devScore
10/10 [==============================] - 1s 34ms/step - loss: 0.6920 - f1_score: 0.6196
concluding trial  3

trainScore
75/75 [==============================] - 8s 98ms/step - loss: 0.6928 - f1_score: 0.6488
devScore
10/10 [==============================] - 1s 62ms/step - loss: 0.6922 - f1_score: 0.6320
concluding trial  4

trainScore
75/75 

,n_convFilters,n_convAndPoolLayers,n_FCLayers,n_NeuronsPerFCLayers,dropoutRate,trainScore,devScore
8,10,5,6,9,0.627213,0.650334,0.650836
9,40,5,7,3,0.872140,0.650164,0.645991
0,29,3,8,8,0.538506,0.649830,0.627677
3,3,5,8,2,0.457297,0.649345,0.619625
1,43,2,2,6,0.854346,0.649321,0.624442
6,16,4,6,9,0.908103,0.649300,0.638469
2,58,3,8,4,0.699733,0.649159,0.638418
4,17,4,3,7,0.860517,0.648768,0.631977
7,19,1,5,9,0.467507,0.648683,0.637643
5,10,3,3,6,0.006308,0.648561,0.645058
